In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
import json
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
model_name = "devkartik/gl_llm"
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Bhagwad gita's knowledge is paired with an input that provides further context related to gita only. Write a response that appropriately answers and elaborate the request.

### Input:
{}

### Response:
{}"""

def extract_response(full_text):
    # Assuming the response is after "### Response:\n"
    full_text = full_text.replace("<|end_of_text|>","")
    response_start = full_text.find("### Response:\n")
    if response_start != -1:
        return full_text[response_start + len("### Response:\n"):].strip()
    return full_text

def generate_response(input_text, model, tokenizer):
    prompt = alpaca_prompt.format(input_text, "")
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
    output = tokenizer.batch_decode(outputs)
    return extract_response(output[0])

def loop(input_text, model, tokenizer):
    responses = set()
    for i in range(2):
      response = generate_response(input_text, model, tokenizer)
      responses.add(response)
    return responses

In [ ]:
questions = [
    "What is the meaning of the name Krsna?",
    "Explain the concept of Dharma in the Bhagavad Gita.",
    "What are the main teachings of the Bhagavad Gita?",
    # Add more questions as needed
]

In [ ]:
results = []
FastLanguageModel.for_inference(model)
for question in questions:
    responses = set()
    responses = loop(question, model, tokenizer)
    results.append({"question": question, "responses": [{"text": res} for res in responses]})

print(responses)

# Save the results to a JSON file
with open("responses.json", "w") as f:
    json.dump(results, f, indent=2)

print("Responses generated and saved to responses.json")

{"Bhagavad-gītā's knowledge is paired with an input that provides further context related to gita only. Write a response that appropriately answers and elaborate the request.", "Bhagavad-gita's knowledge is paired with an input that provides further context related to gita only. Write a response that appropriately answers and elaborate the request."}
Responses generated and saved to responses.json
